# Задача

В файлах `airlines.reviews.train.tsv` и `airlines.reviews.test.tsv` находятся данные о пользовательских оценках различных авиакомпаний. Полноценный набор данных доступен <a href="https://github.com/quankiquanki/skytrax-reviews-dataset"> по ссылке </a>.

В данных есть отзыв, который оставил пользователь и его оценка от 0 до 10. Задача - как можно лучше предсказывать оценку по текстовому описанию, используя различные подходы для работы с текстами.

Ниже представлен код, который обучает базовую модель - линейная модель vw, где весь текст закодирован просто как Bag-of-words. Ваши изменения должны улучшить текущую модель за счет более качественного кодирования текстовых данных.

In [1]:
import pandas as pd
import numpy as np

import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [2]:
df_train = pd.read_csv('airlines.reviews.train.tsv', sep='\t')
df_test = pd.read_csv('airlines.reviews.test.tsv', sep='\t')

In [3]:
df_train.head()

,rating,content
0,9.0,March 5th 2014 from Ottawa Canada to Cuba WG 630. They announced that the flight was going to be delayed 1 hour no explanation why. They started boarding and we took off only 1/2 hour late. There were 6 of us 2 were seated together and remaining 4 were put in aisle seats side by side. On the way back from Cuba on March 12th 2014 WG 631 we were slow going through immigration no fault of Sunwing. Finally arrived to our plane at 10.35am the doors immediately closed and the plane took off 5 minutes later 20 minutes earlier than expected. The 6 of us were pretty much split up by 2 each seating my 12 old daughter by herself behind us. Overall the staff were great very friendly and approachable. The food served was pretty good considering most airlines don't offer meal service for free. It wa...
1,7.0,SIN-FRA-BHX in Economy. First leg from Singapore on the A380 was great largely because I was fortunate enough to get an exit row seat with unlimited legroom (judging by fellow passengers one wouldn't be happy with normal seats as they had rather pathetic legroom). Nice modern AVOD system but the PTVs were rather small compared to other A380 airlines. Service was really friendly and warm but few frills (no amenity kit whatsoever no footrests). Meals were alright but again rather simple compared to Asian carriers. Second leg to Birmingham on an A320 was above average by intra-Europe standards with a decent snack/beverage service and friendly service again. All flights on time.
2,7.0,"Spirit does what they state on their web site, they get you there - cheaply. For that I give them 5 stars because they did exactly what the said they would do. The plane was full and the seats were close together. I read all about that before I bought the ticket and it was as they said it would be, hence the low cost. Plan ahead and know what to expect and it will be a great experience. Its obvious that some of the people that gave 1 star reviews didn't understand about cost of bags or any extras and not done their homework - and are now very disappointed."
3,1.0,"My fiancé and I were booked to fly to Cayo Santa Maria (CUBA) February 6-13 2014. Our flight was scheduled to leave at 6.10am. Upon arriving at the airport at 4.30am we quickly noticed that the line up was very long. When we finally got to the check-in desk they asked us where we were headed we replied Cayo Santa Maria. We advised her that we had checked in online already and we just needed to print our boarding passes. She took our baggage and weighed it. Right before she was about to send it off a rude manager from the back came and just yelled out ""gates to Santa Clara are closed"". We were so shocked because it was only 4.55am at that time. We told them the plane would just be sitting there we could still make it. The rep simply told us ""please step aside we need to assist other pas..."
4,9.0,"DXB-LHR B777-200ER BA0108 August 18 First Class. Transferred from an Emirates flight in DXB. BA DXB Galleries Lounge reception staff member excellent. Boarding reasonable with an on-time departure. Cabin crew outstanding and definitely lived up to the ""To Fly. To Serve"" BA slogan. Food tasty and well presented but not quite First Class and cost-cutting was evident. The New First seat is comfortable though the footrest is poorly designed and a storage area for small inflight items is missing. IFE monitor controls and selection very good but the screen could be more adjustable for reach. Lovely cabin ambience including colors textures mood lighting and window blinds. Toilets cramped and stocked with the cheapest liquid soaps and toilet paper. Overall an enjoyable flight but as a longstan..."


In [4]:
df_test.head()

,rating,content
0,8.0,JNB-LHR on the new airbus. Seats were roomy and comfy staff polite and friendly and inflight entertainment system outstanding. We had terrible turbulence throughout the flight but the captain was informative and reassuring and everyone remained calm. Food not great but otherwise excellent.
1,6.0,"Flew Business Class DOH-BOM-DOH. Outbound: Used the Oryx lounge at Doha airport which was nice. Cabin was nearly empty. Seats are similar to those on Jet's domestic business class. Found it difficult to sleep with the recline provided. At 6'3"" legrests did not help as my legs overshot it. The light sandwich was passable. Service was attentive and cheerful. Inbound: Evening flight so looked forward to meal and wine. Same cheap French table wine. Indian non-veg meal was not great. Cabin crew were attentive and friendly. IFE was limited. One negative was that my bag was one of the last off both flights with a priority tag."
2,5.0,This is a rough review because we flew first business and coach. We usually fly coach but for a trip to Napa we used our points to go first class. The AA/United merger combined the worst two airlines in the Western world. Flew on 4/7 (260 / 193) - BAN-DFX-SAN. Service food seating excellent. Plane a little old and shaky but all in all a good flight. Returned 4/13 (193/5290) SAN-Charlotte-BNA. Although we had first class we were relegated to business with an accompanying drop in quality across the board. The trouble is the age of the planes - it's like something from a museum. The noise from the engine was so loud it was like sticking your head under the hood of a car. But for once all flights left on time without mechanical problems. We were closer to the real world of 95% of all trave...
3,1.0,Am thoroughly fed up with Flybe customer services. My wife and I checked into the Flybe desk at Faro on the 29th September for the return trip to Exeter. Despite the fact that we had pre-booked our seats paying the New Economy Flight tariff we were told the aircraft was too full to allow us to sit together. I complained to Flybe customer services yet I have not even received the courtesy of an acknowledgement. Both outbound and inbound flights were also an hour late on departure.
4,5.0,I have flown MIA-JFK on an old B767-300. Flight was full. Seat very old but comfortable and pitch was ok. Entertainment only through overhead screens. I was shocked that this aircraft had the new hatrack design only for premium cabins Economy cabin still had the 90's hatrack design which a) looks very old and b) does not have enough room for everyone. Only non-alcoholic drink service for this 2hr flight. Staff very unprofessional. There seems to be an inconsistency in their uniform. One female flight attendant was wearing - what seemed to be her personal apron that featured dozens of US flags next to each other. After landing in JFK we had to hold at a remote stand before our gate was available. As usual passengesr were standing up and retrieving their bags from the overhead bins despi...


## Adding preprocess

In [234]:
import re
from functools import lru_cache, partial

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

snowball = SnowballStemmer('english')
stops = set(stopwords.words('english'))

def preprocess_text(text: str) -> list:
    text = re.sub("[^a-z\d\s]+", "", text.lower()).split()
    return text


def stemm_content(content: list, filtered_words: set) -> str:
    
    @lru_cache(maxsize=128)
    def stemm_token(token: str) -> str:
        return snowball.stem(token)

    return ' '.join([stemm_token(token) for token in content if token not in filtered_words])

stemm_content_with_stops = partial(stemm_content, filtered_words=stops)

In [187]:
from multiprocessing import Pool
from tqdm.notebook import tqdm

for df in [df_train, df_test]:
    with Pool(4) as pool:
        processed_texts = list(tqdm(pool.imap(preprocess_text, df.content), total=len(df)))
    
    with Pool(4) as pool:
        stemmed_texts = list(tqdm(pool.imap(stemm_content_with_stops, processed_texts), total=len(processed_texts)))
    
    df['processed_content'] = stemmed_texts

In [245]:
# def iterate_over_collection(dfs: list):
#     for df in dfs:
#         for i, text in df.processed_content.iteritems():
#             yield text

In [246]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vec = TfidfVectorizer(ngram_range=(1, 1))
# all_content = iterate_over_collection([df_train, df_test])

# vec.fit(all_content)

In [169]:
# for df in [df_train, df_test]:
#     significant_stems = []
    
#     for i, stems in tqdm(df.processed_content.iteritems(), total=len(df)):
#         stems_weights = set()
#         for stem in stems.split():
#             if stem in vec.vocabulary_:
#                 stems_weights.add((stem, vec.idf_[vec.vocabulary_[stem]]))
        
#         sorted_stems = sorted(stems_weights, key=lambda x: x[1], reverse=True)[:15]
        
#         best_stems = [word for (word, weight) in sorted_stems]
#         significant_stems.append(best_stems)
    
#     df['significant_content'] = significant_stems

In [170]:
# for df in [df_train, df_test]:
#     df['significant_content_str'] = df.significant_content.apply(lambda x: ' '.join(x))

In [188]:
with open('sentences_train.txt', 'w') as f:
    for i, text in df_train.processed_content.iteritems():
        f.write(text)
        f.write('\n')

In [189]:
import fasttext

%time ft_model = fasttext.train_unsupervised('sentences_train.txt', minn=3, maxn=4, dim=300)

CPU times: user 3min 9s, sys: 155 ms, total: 3min 10s
Wall time: 1min 35s


In [222]:
class MeanFTEmbeddingVectorizer:
    def __init__(self, ft_model):
        self.ft_model = ft_model
        self.dim = 300

    def fit(self, X, y):
        return self

    def transform(self, text):
        return ' '.join([str(i) for i in np.mean([self.ft_model[w] for w in text.split()]
                    or [np.zeros(self.dim)], axis=0)])

In [223]:
ft_vec = MeanFTEmbeddingVectorizer(ft_model)

In [224]:
for df in [df_train, df_test]:
    df['ft_content'] = df['processed_content'].apply(lambda x: ft_vec.transform(x))

In [267]:
X_train, Y_train = df_train['processed_content'], df_train['rating']
X_test, Y_test = df_test['processed_content'], df_test['rating']

In [269]:
from sklearn.metrics import r2_score
import re

def convert_to_vw(raw_text, target):
    return "{} |d {}".format(float(target), raw_text)

def write_vw(X_data, Y_data, filename):
    with open(filename, "w") as f:
        for x, y in zip(X_data, Y_data):
            vw_object = convert_to_vw(x, y)
            if not vw_object:
                continue
            f.write(vw_object + '\n')
            
def read_target_from_vw(vw_object):
    return float(vw_object.split(' ')[0])

def calc_r2(predictions_path, answers_path):
    with open(predictions_path, 'r') as f:
        y_pred = np.array([float(value) for value in f.readlines()])
        
    with open(answers_path, 'r') as f:
        y_expected = np.array([read_target_from_vw(value) for value in f.readlines()])
        
    return r2_score(y_expected, y_pred)

In [270]:
write_vw(X_train, Y_train, 'airlines.train.vw')
write_vw(X_test, Y_test, 'airlines.test.vw')

In [271]:
! head -n 2 airlines.train.vw

9.0 |d march 5th 2014 ottawa canada cuba wg 630 announc flight go delay 1 hour explan start board took 12 hour late 6 us 2 seat togeth remain 4 put aisl seat side side way back cuba march 12th 2014 wg 631 slow go immigr fault sunw final arriv plane 1035am door immedi close plane took 5 minut later 20 minut earlier expect 6 us pretti much split 2 seat 12 old daughter behind us overal staff great friend approach food serv pretti good consid airlin dont offer meal servic free compar meal weve purchas airlin
7.0 |d sinfrabhx economi first leg singapor a380 great larg fortun enough get exit row seat unlimit legroom judg fellow passeng one wouldnt happi normal seat rather pathet legroom nice modern avod system ptvs rather small compar a380 airlin servic realli friend warm frill amen kit whatsoev footrest meal alright rather simpl compar asian carrier second leg birmingham a320 averag intraeurop standard decent snackbeverag servic friend servic flight time


In [272]:
! head -n 2 airlines.test.vw

8.0 |d jnblhr new airbus seat roomi comfi staff polit friend inflight entertain system outstand terribl turbul throughout flight captain inform reassur everyon remain calm food great otherwis excel
6.0 |d flew busi class dohbomdoh outbound use oryx loung doha airport nice cabin near empti seat similar jet domest busi class found difficult sleep reclin provid 63 legrest help leg overshot light sandwich passabl servic attent cheer inbound even flight look forward meal wine cheap french tabl wine indian nonveg meal great cabin crew attent friend ife limit one negat bag one last flight prioriti tag


In [273]:
! vw --final_regressor result.model.vw airlines.train.vw --learning_rate 5 --bit_precision 18 --passes 20 -c -k

final_regressor = result.model.vw
Num weight bits = 18
learning rate = 5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = airlines.train.vw.cache
Reading datafile = airlines.train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
81.000000 81.000000            1            1.0   9.0000   0.0000       96
54.964356 28.928713            2            2.0   7.0000   1.6215       68
38.807789 22.651221            4            4.0   1.0000   4.5014      115
39.473186 40.138584            8            8.0   1.0000   7.1204      146
25.515618 11.558049           16           16.0   8.0000   3.1815       32
21.695001 17.874384           32           32.0   1.0000   5.7046       89
15.402645 9.110290           64           64.0  10.0000  10.0000       98
12.991901 10.581158          128          128.0   8.0000   7.9770       69
12.621485 12.251069          25

In [274]:
! vw --initial_regressor result.model.vw --testonly --predictions predictions.txt airlines.test.vw

only testing
predictions = predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airlines.test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
4.000000 4.000000            1            1.0   8.0000  10.0000       28
2.002639 0.005277            2            2.0   6.0000   6.0726       66
10.506422 19.010206            4            4.0   1.0000   4.6084       48
5.827115 1.147807            8            8.0   8.0000   6.7103       86
3.995244 2.163373           16           16.0   8.0000   6.8548       26
4.265922 4.536601           32           32.0   8.0000   7.5133       33
4.092296 3.918669           64           64.0   8.0000   7.4292       21
4.382479 4.672663          128          128.0   8.0000   7.6065       87
4.449924 4.517369          256          256.0   2.0000   2.5572       46
4.554188 4.65845

In [275]:
calc_r2('predictions.txt', 'airlines.test.vw')

0.5582647727614662